# Versionnage et traçage des données

Nous allons étudier dans ces travaux pratiques comment versionner ses données et conserver des informations d'origine à l'aide de l'outil DVC.

Cet outil découple le versionnage des données et leur stockage.

## Notes sur l'environnement CodeSpace

Vous pouvez exécuter des commandes dans le terminal directement ou dans les cellules de code de ce notebook. Dans ce cas, elles doivent préfixées d'un `!`, elles seraient sinon interprétées comme du Python.

Par ailleurs, si vous avez besoin de copier-coller quelque chose dans le terminal, il faut utiliser ctrl-maj-v et non pas ctrl-v.

## Création du dépôt DagsHub

Créez un compte sur [GitHub](https://github.com) et [DagsHub](https://dagshub.com/) si nécessaire, puis&nbsp;:

Créez ensuite un dépôt vierge sur DagsHub. Une fois le dépôt créé, récupérez votre clef d'accès secrète en cliquant sur «&nbsp;*Get started with Data*&nbsp;» et en la copiant-collant depuis la section «&nbsp;*Connection credentials*&nbsp;».

Remplissez les variables `owner`, `repo` & `token` avec respectivement votre nom d'utilisateur DagsHub, votre nom de dépôt git et votre clef d'accès secrète.

In [ ]:
owner = ""
repo = ""
token = ""

Exécutez maintenant la cellule ci-dessous pour créer une branche distante pointant vers votre dépôt DagsHub (ce qui nous permettra de publier les changements produits pendant ces travaux pratiques pour simuler une tâche réelle).

In [3]:
!git remote add dagshub https://{token}@dagshub.com/{owner}/{repo}.git

## Initialisation du dépôt DVC

Dans le terminal, lancez une commande pour initialiser un dépôt DVC puis publiez les changements sur votre dépôt DagsHub.

### Solution

In [ ]:
!dvc init

In [5]:
!git commit -m "Initialisation de DVC"

[main eef9b59] Initialisation de DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


In [6]:
!git push -u dagshub main

Enumerating objects: 23, done.
Counting objects: 100% (23/23), done.
Delta compression using up to 2 threads
Compressing objects: 100% (20/20), done.
Writing objects: 100% (23/23), 92.85 KiB | 30.95 MiB/s, done.
Total 23 (delta 6), reused 8 (delta 0), pack-reused 0 (from 0)
To https://dagshub.com/m09/dvc-playground.git
 * [new branch]      main -> main
branch 'main' set up to track 'dagshub/main'.


## Récupération d'une première version des données

Il y a plusieurs manières d'importer une source de données dans DVC. L'une d'entre elles est d'utiliser deux commandes&nbsp;: [`dvc get`](https://dvc.org/doc/command-reference/get) d'abord, qui récupère des données depuis un dépôt git/DVC, puis [`dvc add`](https://dvc.org/doc/command-reference/add) qui ajoute les données aux données gérées par DVC.

Après avoir consulté leur aide, utilisez ces deux commandes pour importer le fichier `wiki_movie_plots_deduped.csv` du dépôt git `https://github.com/m09/dataset-wikipedia-movie-plots/` sous le nom `data.csv`.

DVC utilise un dossier de cache qui contient (la plupart du temps) les données utilisées par le projet. Que constatez-vous en observant ce dossier&nbsp;?

N'hésitez pas à observer les fichiers d'extension `.dvc`.

### Solution

In [ ]:
!dvc get https://github.com/m09/dataset-wikipedia-movie-plots wiki_movie_plots_deduped.csv -o data.csv

In [ ]:
!dvc add data.csv

## Ajout du fichier à un commit git

Commitez maintenant ce fichier en suivant les recommandations de DVC données en sortie de la commande `dvc add`, puis publiez ces changements sur DagsHub.

### Solution

In [ ]:
!git add .gitignore data.csv.dvc

In [ ]:
!git commit -m "Ajout d'une première version des données"

In [ ]:
!git tag "v1"

In [ ]:
!git push origin main v1

## Configuration d'un serveur de stockage

DVC peut stocker les données dans divers types de serveurs. DagsHub met à disposition un espace de stockage qui s'utilise comme un bucket s3 (solution de stockage du cloud amazon).

Configurez ce serveur en suivant les instructions disponibles dans l'onglet « *Data* » du bouton vert « *Remote* » de votre dépôt DagsHub puis publiez-y vos données.

### Solution

In [ ]:
!dvc remote add origin s3://dvc
!dvc remote modify origin endpointurl https://dagshub.com/{owner}/{repo}.s3
!dvc remote modify origin --local access_key_id {token}
!dvc remote modify origin --local secret_access_key {token}

In [ ]:
!dvc push -r origin

## Modification des données

Exécutez les cellules suivantes pour modifier le fichier de données&nbsp;:

In [ ]:
!sort -r < data.csv > a && dvc remove data.csv.dvc && mv a data.csv

Utilisez maintenant `dvc add` & `git add` pour enregistrer ce changement.

### Solution

In [ ]:
!dvc add data.csv

In [ ]:
!git add data.csv.dvc .gitignore

In [ ]:
!git commit -m "Données v2"
!git tag "v2"

In [ ]:
!git push origin main v2

In [ ]:
!dvc push -r origin

## Retour aux données originales

On imagine avoir détecté un problème avec nos nouvelles données&nbsp;: on souhaite revenir à la première version. Utilisez `git checkout` pour revenir à la version antérieure des métadonnées, puis `dvc checkout` pour récupérer le fichier correspondant.

Commitez maintenant ce retour à la première version.

### Solution

In [ ]:
!git checkout v1 data.csv.dvc

In [ ]:
!dvc checkout data.csv

In [ ]:
!git add data.csv.dvc

In [ ]:
!git commit -m "Retour aux données v1"
!git tag v3

In [ ]:
!git push origin main v3

In [ ]:
!dvc push -r origin

## Traitement des données

Exécutez la cellule suivante qui contient un script Python qui prend deux arguments, et qui écrit au chemin donné par le deuxième argument le contenu du fichier au chemin donné par le premier argument, en passant le contenu en majuscule.

Cette cellule va écrire ce script au chemin `upper.py`

In [1]:
%%writefile upper.py
from pathlib import Path
from sys import argv

Path(argv[2]).write_text(
    Path(argv[1]).read_text(encoding="utf8").upper(),
    encoding="utf8")

Writing upper.py


Ajoutez une étape de traitement avec [`dvc stage add`](https://dvc.org/doc/command-reference/stage/add) qui prend en entrée le fichier `data.csv` et produit le fichier `data-upper.csv` à partir de ce script.

Exécutez maintenant cette étape de traitement avec [`dvc repro`](https://dvc.org/doc/command-reference/repro) puis sauvegardez les données dans le serveur de stockage et les métadonnées dans git.

### Solution

In [ ]:
!dvc stage add -n transform-uppercase -d data.csv -o data-upper.csv python upper.py data.csv data-upper.csv

In [ ]:
!dvc repro

In [ ]:
!git add dvc.yaml .gitignore dvc.lock

In [ ]:
!git commit -m "Pipeline pour passer un fichier en majuscule"

In [ ]:
!git push origin main

In [ ]:
!dvc push -r origin